# **Plan para Entrenar el Modelo de Recomendación**

## **1. Estrategia del Modelo**
Según el contexto, el modelo debe:
- Recomendar 5 productos por sesión (`session_id`).
- Considerar usuarios logueados y no logueados.
- Incorporar datos de usuarios, productos e interacciones.

Podemos optar por un enfoque híbrido:
1. **Colaborativo:** Basado en interacciones previas entre usuarios y productos.
2. **Basado en contenido:** Usando características como embeddings, clústeres y categorías.
3. **Popularidad:** Para usuarios nuevos o sin historial.

---

## **2. Flujo del Entrenamiento**
1. **Preparar los Datos:**
   - Dividir `train_data` en entrenamiento y prueba.
   - Asegurar que cada sesión tenga 5 recomendaciones.

2. **Entrenamiento del Modelo:**
   - Construir un pipeline que combine los enfoques colaborativo, basado en contenido y de popularidad.
   - Usar embeddings (`reduced_embedding`) y datos de usuarios (`RFM_score`, `user_class`).

3. **Evaluación del Modelo:**
   - Usar la métrica **NDCG** para medir la calidad de las recomendaciones.

4. **Predicción:**
   - Generar recomendaciones para todas las sesiones en el conjunto de prueba.


---

## **Paso 1: Preparar los Datos**

#### **1. Dividir `train_data`**
Dividiremos el dataset en conjuntos de **entrenamiento** y **prueba**:
- **Entrenamiento:** 80% de las sesiones.
- **Prueba:** 20% de las sesiones.


In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd

# Cargar el dataset enriquecido
train_data_path = '/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/train_data_enriched.pkl'
train_data = pd.read_pickle(train_data_path)

# Dividir en conjuntos de entrenamiento y prueba
train_sessions, test_sessions = train_test_split(
    train_data['session_id'].unique(), test_size=0.2, random_state=42
)

# Filtrar las interacciones de entrenamiento y prueba
train_set = train_data[train_data['session_id'].isin(train_sessions)]
test_set = train_data[train_data['session_id'].isin(test_sessions)]

# Verificar tamaños
print(f"Total sesiones de entrenamiento: {len(train_sessions)}")
print(f"Total sesiones de prueba: {len(test_sessions)}")
print(f"Interacciones en conjunto de entrenamiento: {len(train_set)}")
print(f"Interacciones en conjunto de prueba: {len(test_set)}")


Total sesiones de entrenamiento: 3659004
Total sesiones de prueba: 914752
Interacciones en conjunto de entrenamiento: 37217837
Interacciones en conjunto de prueba: 9333608


## **Paso 2: Verificar la División**
Revisaremos las distribuciones de las columnas clave (`add_to_cart`, `user_id`, `partnumber`) para asegurarnos de que las sesiones están bien distribuidas entre los conjuntos.


In [2]:
# Distribución en el conjunto de entrenamiento
print(train_set['add_to_cart'].value_counts(normalize=True))
print(train_set['user_id'].value_counts(normalize=True).head())

# Distribución en el conjunto de prueba
print(test_set['add_to_cart'].value_counts(normalize=True))
print(test_set['user_id'].value_counts(normalize=True).head())


add_to_cart
0    0.941039
1    0.058961
Name: proportion, dtype: float64
user_id
-1         0.852830
 436999    0.000092
 304018    0.000065
 88890     0.000057
 31534     0.000051
Name: proportion, dtype: float64
add_to_cart
0    0.940926
1    0.059074
Name: proportion, dtype: float64
user_id
-1         0.852214
 184051    0.000222
 540020    0.000145
 43208     0.000142
 124268    0.000138
Name: proportion, dtype: float64


### **Revisión de los Datos**

#### **1. División en Conjuntos**
- **Sesiones:**
  - Entrenamiento: **3,659,004** sesiones.
  - Prueba: **914,752** sesiones.
- **Interacciones:**
  - Entrenamiento: **37,217,837** interacciones.
  - Prueba: **9,333,608** interacciones.

La proporción entre los conjuntos de entrenamiento (80%) y prueba (20%) está correctamente balanceada.

---

#### **2. Distribución de `add_to_cart`**
- **Entrenamiento:** 
  - No añadido al carrito (`0`): **94.10%**.
  - Añadido al carrito (`1`): **5.90%**.
- **Prueba:**
  - No añadido al carrito (`0`): **94.09%**.
  - Añadido al carrito (`1`): **5.91%**.

La distribución de la variable objetivo `add_to_cart` es consistente entre los conjuntos de entrenamiento y prueba.

---

#### **3. Distribución de `user_id`**
- La mayoría de las interacciones son de usuarios no logueados (`user_id = -1`), lo cual es consistente con el contexto del problema:
  - **Entrenamiento:** 85.28% de las interacciones.
  - **Prueba:** 85.22% de las interacciones.

Los usuarios logueados representan una minoría, pero las proporciones son similares en ambos conjuntos.


## **Paso 3: Guardar los conjuntos de entrenamiento**

In [3]:
# Guardar en formato Pickle
train_set.to_pickle('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/train_set.pkl')
test_set.to_pickle('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/test_set.pkl')

# Guardar en formato CSV
train_set.to_csv('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/train_set.csv', index=False)
test_set.to_csv('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/test_set.csv', index=False)


---

# Plan para el Modelo Híbrido

## 1. Componentes del Modelo

El modelo incluirá tres enfoques complementarios:

### Basado en Popularidad:
- Recomendaciones de los productos más populares (por ejemplo, en la misma categoría o clúster).
- Ideal para usuarios no logueados o sin historial.

### Basado en Contenido:
- Utiliza las características del producto (`reduced_embedding`, `cluster`, `discount_category`, etc.) y del usuario (`RFM_score`, `region`, etc.).
- Perfecto para recomendaciones personalizadas según preferencias.

### Filtrado Colaborativo:
- Basado en patrones de interacción entre usuarios y productos (`add_to_cart`).
- Útil para capturar tendencias entre usuarios con comportamientos similares.

---

## 2. Arquitectura del Pipeline

### Input:
- **Usuario:**
  - `user_id`, `RFM_score`, etc.
- **Productos:**
  - `partnumber`, `cluster`, `embedding`.
- **Historial de interacciones:**
  - `add_to_cart`, `session_id`.

### Procesamiento:

- **Para usuarios logueados:**
  - Filtrado colaborativo + contenido + popularidad.
- **Para usuarios no logueados:**
  - Popularidad + contenido.

### Output:
- Recomendación de los 5 productos más relevantes por sesión.

---

## 3. Algoritmos Seleccionados

### Basado en Popularidad:
- Ranking simple por frecuencia (popularidad).

### Basado en Contenido:
- Similaridad coseno con embeddings (`reduced_embedding`).

### Filtrado Colaborativo:
- **Matrix Factorization (SVD):** Para capturar relaciones usuario-producto.
- **Alternativa:** Modelos como **ALS (Alternating Least Squares)** si trabajamos con datos dispersos.



---

### **Paso 1: Función para Popularidad**
Esta función recomendará los productos más populares dentro de un clúster, categoría o en general.

In [2]:
def recommend_by_popularity(train_data, cluster=None, top_n=5):
    """
    Recomienda productos basados en popularidad.
    Si se especifica un clúster, filtra por clúster; de lo contrario, usa la popularidad general.
    """
    if cluster is not None:
        # Filtrar por clúster
        popular_products = (
            train_data[train_data['cluster'] == cluster]
            .groupby('partnumber')['popularity']
            .sum()
            .reset_index()
            .sort_values('popularity', ascending=False)
        )
    else:
        # Popularidad general
        popular_products = (
            train_data.groupby('partnumber')['popularity']
            .sum()
            .reset_index()
            .sort_values('popularity', ascending=False)
        )
    
    # Devolver los Top N productos
    return popular_products['partnumber'].head(top_n).tolist()


### **Paso 2: Función para Similaridad Basada en Contenido**
Esta función utiliza los embeddings de productos para calcular similitudes y recomendar productos similares.

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def recommend_by_content(product_embeddings, target_product, top_n=5):
    """
    Recomienda productos basados en similitud de embeddings.
    """
    # Calcular similitudes
    similarity_matrix = cosine_similarity(product_embeddings)
    
    # Buscar índice del producto objetivo
    product_index = target_product
    
    # Obtener los índices más similares
    similar_indices = np.argsort(similarity_matrix[product_index])[::-1][1:top_n + 1]
    
    # Devolver los productos similares
    return similar_indices.tolist()


### **Paso 3: Filtrado Colaborativo**
Esta función implementa un enfoque basado en factorización de matrices (SVD) para recomendaciones personalizadas.

In [6]:
import pandas as pd
# Cargar train_data desde archivo guardado

train_data_path = '/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/train_set.pkl'
train_data = pd.read_pickle(train_data_path)

In [7]:
from surprise import SVD, Dataset, Reader



def train_collaborative_filtering(train_sample):
    """
    Entrena un modelo de filtrado colaborativo (SVD) utilizando una submuestra.
    """
    # Preparar datos para Surprise
    reader = Reader(rating_scale=(0, 1))
    data = Dataset.load_from_df(
        train_sample[['user_id', 'partnumber', 'add_to_cart']],
        reader
    )
    trainset = data.build_full_trainset()

    # Entrenar modelo SVD
    model = SVD()
    model.fit(trainset)
    return model

# Submuestrear el dataset (10% de las interacciones)
train_sample = train_data.sample(frac=0.1, random_state=42)

# Entrenar el modelo con la submuestra
print("Entrenando modelo con submuestra...")
cf_model = train_collaborative_filtering(train_sample)
print("Modelo entrenado.")


Entrenando modelo con submuestra...
Modelo entrenado.


In [8]:
def recommend_by_collaborative(model, user_id, product_list, top_n=5):
    """
    Recomienda productos usando filtrado colaborativo entrenado.
    """
    # Predecir puntuaciones para cada producto en la lista
    predictions = [
        (product, model.predict(user_id, product).est) for product in product_list
    ]
    # Ordenar productos por puntuación en orden descendente
    predictions = sorted(predictions, key=lambda x: x[1], reverse=True)
    # Retornar los Top N productos recomendados
    return [pred[0] for pred in predictions[:top_n]]


### **Paso 4: Integrar el Pipeline Híbrido**
- Una vez implementadas estas funciones, podemos integrarlas en un pipeline híbrido para combinar recomendaciones de popularidad, contenido y colaborativo.
- Vamos a integrar las funciones en un pipeline híbrido que combine los enfoques de popularidad, contenido, y filtrado colaborativo. 
- Este pipeline priorizará las recomendaciones dependiendo de la información disponible para cada usuario.

### **Pipeline Híbrido**
1. **Usuarios Logueados:**
   - **Filtrado Colaborativo:** Si existe historial.
   - **Basado en Contenido:** Si no hay historial suficiente.
   - **Popularidad:** Para completar las recomendaciones.

2. **Usuarios No Logueados:**
   - **Basado en Popularidad:** Principal fuente de recomendaciones.
   - **Basado en Contenido:** Como complemento, si hay información contextual.


In [8]:
class HybridRecommender:
    def __init__(self, train_data, products_data):
        self.train_data = train_data
        self.products_data = products_data
        
        # Entrenar modelo de filtrado colaborativo
        print("Entrenando modelo de filtrado colaborativo...")
        self.cf_model = train_collaborative_filtering(train_data.sample(frac=0.1, random_state=42))
        print("Modelo de filtrado colaborativo entrenado.")
        
        # Obtener embeddings de productos
        self.product_embeddings = np.stack(products_data['reduced_embedding'])
    
    def recommend(self, user_id, session_id, top_n=5):
        """
        Genera recomendaciones híbridas para un session_id.
        """
        # Generar recomendaciones por usuario
        if user_id != -1:
            recommendations = recommend_by_collaborative(
                self.cf_model,
                user_id,
                self.products_data['partnumber'].tolist(),
                top_n=top_n
            )
        else:
            # Usuario no logueado
            recommendations = recommend_by_popularity(self.train_data, top_n=top_n)
        
        return recommendations


Probar el Pipeline

In [9]:
import pandas as pd

# Ruta al archivo de products_data enriquecido
products_data_path = '/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/products_data_enriched.pkl'

# Cargar el dataset
products_data = pd.read_pickle(products_data_path)

# Verificar las primeras filas
print(products_data.head())

   discount                                          embedding  partnumber  \
0         0  [-0.13401361, -0.1200429, -0.016117405, -0.167...      -32760   
1         0  [-0.0949274, -0.107294075, -0.16559914, -0.174...      -24105   
2         0  [-0.12904441, -0.07724628, -0.09799071, -0.164...      -26117   
3         1  [-0.12783332, -0.133868, -0.10101265, -0.18888...      -29449   
4         1  [-0.14092924, -0.1258284, -0.10809927, -0.1765...      -31404   

   color_id  cod_section  family  popularity discount_category  \
0        85            4      73           0        Full Price   
1       135            4      73           0        Full Price   
2       339            4      73           0        Full Price   
3       135            4      73           0        Discounted   
4         3            4      73           0        Discounted   

                                   reduced_embedding  cluster  \
0  [1.2056737, 4.4139566, 0.2004558, -1.7360026, ...       18   
1  [

In [10]:
# Lista de productos disponibles
product_list = train_data['partnumber'].unique()

# Probar recomendaciones para un usuario logueado
user_id = 436999  # Usuario logueado de ejemplo
recommendations = recommend_by_collaborative(cf_model, user_id, product_list, top_n=5)

print(f"Recomendaciones para usuario {user_id}: {recommendations}")


Recomendaciones para usuario 436999: [25438, 25026, 14539, 6982, 20347]


In [13]:
# # Inicializar el pipeline híbrido
# recommender = HybridRecommender(train_data, products_data)

# # Probar recomendaciones para un usuario logueado
# user_id = 436999  # Usuario logueado de ejemplo
# print("Recomendaciones para usuario logueado:")
# print(recommender.recommend(user_id, top_n=5))

# # Probar recomendaciones para un usuario no logueado
# user_id = -1  # Usuario no logueado
# print("\nRecomendaciones para usuario no logueado:")
# print(recommender.recommend(user_id, top_n=5))


---

##  Generar `predictions_3.json` -> 51447 lines

In [ ]:
# import json

# # Inicializar el pipeline
# recommender = HybridRecommender(train_data, products_data)

# # Generar recomendaciones para todas las session_id
# session_recommendations = {}

# print("Generando recomendaciones para todas las session_id...")

# for session_id, group in test_set.groupby('session_id'):
#     # Obtener el user_id asociado a la sesión
#     user_id = group['user_id'].iloc[0]

#     # Generar recomendaciones usando el pipeline híbrido
#     recommendations = recommender.recommend(user_id, session_id, top_n=5)

#     # Eliminar duplicados en las recomendaciones
#     unique_recommendations = list(dict.fromkeys(recommendations))

#     # Guardar recomendaciones para la sesión
#     session_recommendations[str(session_id)] = unique_recommendations[:5]

# # Verificar que todas las session_id estén cubiertas
# assert len(session_recommendations) == test_set['session_id'].nunique(), (
#     "No todas las session_id tienen recomendaciones."
# )

# # Guardar en predictions_3.json con el formato correcto
# output_data = {"target": session_recommendations}
# output_path = '/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/predictions/predictions_3.json'
# with open(output_path, 'w') as f:
#     json.dump(output_data, f, indent=4)

# print(f"Archivo predictions_3.json generado en: {output_path}")


: 

In [1]:
import json
import multiprocessing as mp
from functools import partial
from tqdm import tqdm
import psutil

# Inicializar el pipeline
recommender = HybridRecommender(train_data, products_data)

def process_sessions(session_chunk, recommender):
    """
    Procesa un subconjunto de session_id y genera recomendaciones.
    """
    chunk_recommendations = {}
    for session_id, group in session_chunk:
        user_id = group['user_id'].iloc[0]
        recommendations = recommender.recommend(user_id, session_id, top_n=5)
        unique_recommendations = list(dict.fromkeys(recommendations))
        chunk_recommendations[str(session_id)] = unique_recommendations[:5]
    return chunk_recommendations

# Dividir las sesiones en bloques
def split_sessions(test_set, n_chunks):
    """
    Divide las session_id en n_chunks.
    """
    session_groups = list(test_set.groupby('session_id'))
    chunk_size = max(len(session_groups) // n_chunks, 100)  # Bloques de al menos 100 sesiones
    return [session_groups[i:i + chunk_size] for i in range(0, len(session_groups), chunk_size)]

# Monitoreo del uso de CPU
def monitor_cpu_usage(interval=5):
    """
    Monitorea el uso del CPU y la memoria en intervalos de tiempo.
    """
    print("Monitoreando uso del CPU y memoria...")
    while True:
        cpu_usage = psutil.cpu_percent(interval=interval)
        memory_usage = psutil.virtual_memory().percent
        print(f"Uso del CPU: {cpu_usage}% | Uso de Memoria: {memory_usage}%")

# Configurar paralelismo
n_processes = mp.cpu_count()  # Usar todos los núcleos disponibles
session_chunks = split_sessions(test_set, n_processes)

print(f"Dividiendo {len(test_set['session_id'].unique())} sesiones en {len(session_chunks)} bloques para procesamiento paralelo...")

# Procesar las sesiones en paralelo con monitoreo de progreso
with mp.Pool(n_processes) as pool:
    results = list(tqdm(pool.imap(partial(process_sessions, recommender=recommender), session_chunks), 
                        total=len(session_chunks), desc="Procesando bloques"))

# Combinar resultados
session_recommendations = {k: v for chunk in results for k, v in chunk.items()}

# Verificar que todas las session_id estén cubiertas
assert len(session_recommendations) == test_set['session_id'].nunique(), (
    "No todas las session_id tienen recomendaciones."
)


NameError: name 'HybridRecommender' is not defined

In [ ]:
# Guardar en predictions_3.json con el formato correcto
output_data = {"target": session_recommendations}
output_path = '/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/predictions/predictions_3.json'
with open(output_path, 'w') as f:
    json.dump(output_data, f, indent=4)

print(f"Archivo predictions_3.json generado en: {output_path}")


---


#### **1. Evaluar el Modelo de Filtrado Colaborativo**
Podemos evaluar el modelo usando la métrica **NDCG** para medir la calidad de las recomendaciones en el conjunto de prueba.



#### **2. Integrar Filtrado Colaborativo con Popularidad y Contenido**
Ahora que tenemos un modelo funcional para filtrado colaborativo, podemos:
- Combinarlo con las recomendaciones basadas en **popularidad** y **contenido**.
- Probar el pipeline híbrido para usuarios logueados y no logueados.

In [10]:
products_data = pd.read_pickle('/home/pablost/Hackathon_inditex_data_science/hackathon-inditex-data-recommender/data/processed/new_processed/products_data.pkl')

print(products_data.columns)


Index(['discount', 'embedding', 'partnumber', 'color_id', 'cod_section',
       'family'],
      dtype='object')
